In [ ]:
# Import dependencies
import pyomo.environ as pyo
import pandas as pd
import numpy as np
import time as tm
import os.path
import random as rnd
import copy
import string

from openpyxl import load_workbook
from openpyxl.utils.cell import range_boundaries
from grid_generator_latest import generate_grid
from collections import Counter

In [ ]:
# Record time checkpoints
# Requires global variable: Checkpoints = []
def Timer(Point):   # String label for current checkpoint
    Checkpoints.append([Point, tm.perf_counter()])
    
# Output list of checkpoint labels and times
def WriteCheckpoints():
    print('\nCheckpoint    Seconds')
    print('---------------------')
    Start = Checkpoints[0][1]
    for i in range(1, len(Checkpoints)):
        Point = Checkpoints[i][0]
        TimeStep = Checkpoints[i][1] - Start
        print(f'{Point:12}{TimeStep:9,.1f}')
        
# Generic loader from Excel file, given worksheet and named range
def LoadFromExcel(ExcelFile, Worksheet, Range):
    wb = load_workbook(filename=ExcelFile, read_only = True, data_only = True)
    ws = wb[Worksheet]
    dests = wb.defined_names[Range].destinations
    for title, coord in dests:
        min_col, min_row, max_col, max_row = range_boundaries(coord)
        data = ws.iter_rows(min_row, max_row, min_col, max_col, values_only = True)
    df = pd.DataFrame(data)
    return df

# Write model to file, if required. The format will be inferred by Pyomo from the file extension, e.g. .gams or .nl
def WriteModelToFile(WriteFile, Model):
    if WriteFile:
        Model.write(ModelFile, io_options={'symbolic_solver_labels': False})   # symbolic_solver_labels of True is easier to read, but a longer file

In [ ]:
# Create the Solver object for either NEOS or a local solver
def SetUpSolver(Model):
    Model.Options = None
    if Neos:
        Solver = pyo.SolverManagerFactory('neos')   # Solver on NEOS
        if pyo.value(Model.Engine) == 'cplex':   # Linear
            Model.Options = {'timelimit': Model.TimeLimit, 'mipgap': MipGap}
        elif pyo.value(Model.Engine) == 'octeract':   # Linear or non-linear
            Model.Options = {'MAX_SOLVER_TIME': Model.TimeLimit, 'MILP_SOLVER': 'HIGHS'}
        elif pyo.value(Model.Engine) == 'couenne':   # Non-linear
            print('No options for Couenne')
        else:
            print('Unknown NEOS solver when setting options')
    else:
        Solver = pyo.SolverFactory(pyo.value(Model.Engine))   # Local solver installed
        if pyo.value(Model.Engine) == 'couenne':   # Non-linear
            print('No options for Couenne') # Couenne doesn't accept command line options, use couenne.opt instead
        elif pyo.value(Model.Engine) == 'cbc':   # Linear
            Solver.options['seconds'] = pyo.value(Model.TimeLimit)
            Solver.options['log'] = 1   # Default is 1. 2 or 3 provides more detail. Also have slog, which provides much more detail
        elif pyo.value(Model.Engine) == 'appsi_highs':   # Linear
            Solver.options['time_limit'] = pyo.value(Model.TimeLimit)
            Solver.options['log_file'] = 'highs.log'   # Sometimes HiGHS doesn't update the console as it solves, so write log file too
            Solver.options['mip_rel_gap'] = MipGap   # Relative gap. 10 = stop at first feasible solution. 0 = find optimum. 0.1 = within 10% of optimum
            #Solver.options['parallel'] = 'on'
            #Solver.options['mip_heuristic_effort'] = 0.2   # default = 0.05, range = 0..1
        elif pyo.value(Model.Engine) == 'gurobi_direct':
            Solver.options['timelimit'] = pyo.value(Model.TimeLimit)
            Solver.options['logfile'] = 'gurobi.log'
            Solver.options['mipgap'] = MipGap
            Solver.options['solutionlimit'] = SolutionLimit
            #Solver.options['heuristics'] = 0.25   # 0..1 default = 0.05
        else:
            print('Unknown local solver when setting options')
    
    return Solver, Model

# Call either NEOS or a local solver
def CallSolver(Solver, Model):
    if Neos:
        if Model.Options == None:
            Results = Solver.solve(Model, load_solutions = LoadSolution, tee = Verbose, solver = Model.Engine)
        else:
            Results = Solver.solve(Model, load_solutions = LoadSolution, tee = Verbose, solver = Model.Engine, options = Model.Options)
    else:
        Results = Solver.solve(Model, load_solutions = LoadSolution, tee = Verbose)
    
    return Results, Model

In [ ]:
# Load data from Excel file
def GetData(WordFile, WordWorksheet, GridFile, GridWorksheet):
    Rank = LoadFromExcel(WordFile, WordWorksheet, 'rank')
    Frequency = LoadFromExcel(WordFile, WordWorksheet, 'frequency')
    Word = LoadFromExcel(WordFile, WordWorksheet, 'word')
    Rank.columns = ['Candidate']
    Frequency.columns = ['Candidate']
    Word.columns = ['Candidate']
    
    GridWords = LoadFromExcel(GridFile, GridWorksheet, 'NumWords')
    AcrossRef = LoadFromExcel(GridFile, GridWorksheet, 'AcrossRef')
    AcrossPos = LoadFromExcel(GridFile, GridWorksheet, 'AcrossPos')
    DownRef = LoadFromExcel(GridFile, GridWorksheet, 'DownRef')
    DownPos = LoadFromExcel(GridFile, GridWorksheet, 'DownPos')
    
    return Rank, Frequency, Word, GridWords, AcrossRef, AcrossPos, DownRef, DownPos

# Define model data, assigning all data to the Model
def DefineModelData(Model, Rank, Frequency, Word, GridWords, AcrossRef, AcrossPos, DownRef, DownPos):
    Size = SampleSize
    if Size == 0:
        Size = len(Rank)
    else:
        Size = min(SampleSize, len(Rank))
    print(f'Lexicon size: {Size:,.0f}')
    
    Model.Candidate = pyo.Set(initialize = range(0, Size))   # Set of candidate words 
    Model.Ascii = pyo.Set(initialize = range(0, MaxWordLength))   # Set of ASCII character codes in a word
    Model.Rank = pyo.Param(Model.Candidate, within = pyo.NonNegativeIntegers, mutable = True)   # Lexicon rank
    Model.Frequency = pyo.Param(Model.Candidate, within = pyo.NonNegativeReals, mutable = True)   # Lexicon frequency
    Model.Length = pyo.Param(Model.Candidate, within = pyo.NonNegativeIntegers, mutable = True)   # Number of characters in each candidate word
    Model.Word = pyo.Param(Model.Candidate, Model.Ascii, within = pyo.NonNegativeIntegers, mutable = True)   # Each candidate word, split into ASCII codes

    List = [i for i in range(0, len(Rank))]   # Row numbers for whole lexicon
    if Size == len(Rank):
        Sample = List   # Use whole lexicon
    else:
        Sample = rnd.sample(List, Size)   # Use sample of lexicon
    CandidateNum = 0
    for c in range(0, len(Rank)):   # Populate data for selected sample words
        if c in Sample:
            Model.Rank[CandidateNum] = Rank['Candidate'][c]
            Model.Frequency[CandidateNum] = Frequency['Candidate'][c]
            Model.Length[CandidateNum] = len(Word['Candidate'][c])
            LettersAscii = list(bytes(Word['Candidate'][c], 'ascii'))   # ASCII codes for word's characters
            for a in Model.Ascii:   # list of ASCII codes for each letter of a word, padded with zeroes beyond the word's length
                if a >= len(LettersAscii):
                    Model.Word[CandidateNum, a] = 0
                else:
                    Model.Word[CandidateNum, a] = LettersAscii[a]
            CandidateNum += 1
                
    Grid_rows, Grid_cols = np.shape(AcrossRef)   # Note: Model defined as h, w while data accessed via w, h
        
    Model.GridWords = pyo.Set(initialize = range(0, GridWords.iloc[0][0]))   # Set for number of words in the grid
    Model.GridWidth = pyo.Set(initialize = range(0, Grid_cols))   # Set width of the grid
    Model.GridHeight = pyo.Set(initialize = range(0, Grid_rows))   # Set for the height of the grid
    Model.AcrossRef = pyo.Param(Model.GridHeight, Model.GridWidth, within = pyo.NonNegativeIntegers, mutable = True)
    Model.AcrossPos = pyo.Param(Model.GridHeight, Model.GridWidth, within = pyo.NonNegativeIntegers, mutable = True)
    Model.DownRef = pyo.Param(Model.GridHeight, Model.GridWidth, within = pyo.NonNegativeIntegers, mutable = True)
    Model.DownPos = pyo.Param(Model.GridHeight, Model.GridWidth, within = pyo.NonNegativeIntegers, mutable = True)
    Model.GridLengths = pyo.Param(Model.GridWords, within = pyo.NonNegativeIntegers, mutable = True, initialize = 0)

    for w in Model.GridWidth:
        for h in Model.GridHeight:
            Model.AcrossRef[h, w] = AcrossRef[w][h]   # Populate grid encoding
            Model.AcrossPos[h, w] = AcrossPos[w][h]
            Model.DownRef[h, w] = DownRef[w][h]
            Model.DownPos[h, w] = DownPos[w][h]
            if AcrossRef[w][h] >= 1:   # Get length of "across" words by looking at maximum position of each word
                Model.GridLengths[AcrossRef[w][h] - 1] = max(pyo.value(Model.GridLengths[AcrossRef[w][h] - 1]), AcrossPos[w][h])
            if DownRef[w][h] >= 1:   # Get length of "down" words by looking at maximum position of each word
                Model.GridLengths[DownRef[w][h] - 1] = max(pyo.value(Model.GridLengths[DownRef[w][h] - 1]), DownPos[w][h])

In [ ]:
# Define model
def DefineModel(Model):
    Model.Allocation = pyo.Var(Model.Candidate, Model.GridWords, within = pyo.Binary, initialize = 0)   # Allocate candidate words to grid

    def rule_PosOnce(Model, g):   # Allocate exactly one candidate to each grid word position
        return sum(Model.Allocation[c, g] for c in Model.Candidate) == 1
    Model.EachPositionOnce = pyo.Constraint(Model.GridWords, rule = rule_PosOnce)
    
    def rule_WordOnce(Model, c):   # Allocate each word to a grid position at most once. Optional constraint
        return sum(Model.Allocation[c, g] for g in Model.GridWords) <= 1
    Model.EachWordOnce = pyo.Constraint(Model.Candidate, rule = rule_WordOnce)

    def rule_Fit(Model, g):   # Ensure word exactly fills its allocated grid space
        return sum(Model.Allocation[c, g] * Model.Length[c] for c in Model.Candidate) == Model.GridLengths[g]
    Model.WordsFit = pyo.Constraint(Model.GridWords, rule = rule_Fit)

    def rule_Intersection(Model, g1, g2, w, h):   # The intersection of grid words must have the same letter
        if pyo.value(Model.AcrossRef[w, h]) != 0 and pyo.value(Model.DownRef[w, h]) != 0 and g1 == pyo.value(Model.AcrossRef[w, h]) - 1 \
                and g2 == pyo.value(Model.DownRef[w, h]) - 1:
            return sum(Model.Allocation[c, g1] * Model.Word[c, pyo.value(Model.AcrossPos[w, h]) - 1] for c in Model.Candidate) == \
                sum(Model.Allocation[c, g2] * Model.Word[c, pyo.value(Model.DownPos[w, h]) - 1] for c in Model.Candidate)
        else:
            return pyo.Constraint.Skip
    Model.Crossover = pyo.Constraint(Model.GridWords, Model.GridWords, Model.GridWidth, Model.GridHeight, rule = rule_Intersection)

    def rule_Obj(Model):
        return sum(sum(Model.Allocation[c, g] for g in Model.GridWords) * Model.Frequency[c] for c in Model.Candidate)
    Model.Obj = pyo.Objective(rule = rule_Obj, sense = pyo.maximize)

In [ ]:
def find_indices(lst, element):
    indices = []
    start_index = None

    for i, value in enumerate(lst):
        if value == element:
            if start_index is None:
                start_index = i
        elif start_index is not None:
            indices.append((start_index, i - 1))
            start_index = None

    if start_index is not None:
        indices.append((start_index, len(lst) - 1))

    return indices

def get_grid_data_MILP(size = 7, display_grid = False):
    json_data = generate_grid(grid_size = size, b_factor = 6, display_grid = display_grid)
    rows, cols = json_data['size']['rows'], json_data['size']['cols']
    grid_2d = ['|'.join(json_data['grid'][i:i+cols]).replace('.', '0').split('|') for i in range(0, len(json_data['grid']), rows)]
    grid_2d = [[0 if col == '0' else -1 for col in row] for row in grid_2d]
    
    across_ref_ = copy.deepcopy(grid_2d)
    across_pos_ = copy.deepcopy(grid_2d)
    down_ref_ = copy.deepcopy(grid_2d)
    down_pos_ = copy.deepcopy(grid_2d)

    ref_count = 1
    pos_count = 1
    for i, row in enumerate(grid_2d):
        indices = find_indices(row, -1)
        for index_tuple in indices:
            pos_count = 1
            for j in range(index_tuple[0], index_tuple[1] + 1):
                across_ref_[i][j] = ref_count
                across_pos_[i][j] = pos_count
                pos_count += 1
            ref_count += 1

    for i in range(len(grid_2d)):
        for j in range(len(grid_2d[0])):
            if down_ref_[i][j] == -1:
                pos_count = 1
                for k in range(i, len(grid_2d)):
                    if down_ref_[k][j] != 0:
                        down_ref_[k][j] = ref_count
                        down_pos_[k][j] = pos_count
                        pos_count += 1
                    elif down_ref_[k][j] == 0:
                        break
                ref_count += 1

    return pd.DataFrame(across_ref_), pd.DataFrame(across_pos_), pd.DataFrame(down_ref_), pd.DataFrame(down_pos_), pd.DataFrame([ref_count-1])

def get_word_file_MILP(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        words = file.read().split()

    translator = str.maketrans('', '', string.punctuation)
    cleaned_words = [word.translate(translator).lower() for word in words]

    char_frequencies = Counter(''.join(cleaned_words))

    def word_score(word):
        return sum(char_frequencies[char] for char in word) / len(word)

    word_scores = {word: word_score(word) for word in cleaned_words}

    frequency_list = []
    word_list = []
    for word, score in sorted(word_scores.items(), key=lambda x: x[1], reverse=True):
        word_list.append(word)
        frequency_list.append(score)

    Rank = pd.DataFrame(range(1, len(word_list) + 1), columns = ['Candidate'])
    Word = pd.DataFrame(word_list, columns = ['Candidate'])
    Frequency = pd.DataFrame(frequency_list, columns = ['Candidate'])
    return Rank, Word, Frequency

In [ ]:
def Main():
    Timer('Start');
    Finished = False
    Iteration = 0
    Seed = RandomSeed

    while not Finished:
        rnd.seed(Seed)
        Model = pyo.ConcreteModel(name = ModelName)
        Model.Engine = SolverName
        Model.TimeLimit = TimeLimit
        
        if Iteration == 0:
            print(Model.name)   # Print the model name only once
            print('Grid:    ', GridFile.rsplit('\\', 1)[-1])
            print('Lexicon: ', WordFile.rsplit('\\', 1)[-1])
        print('\nIteration: ', Iteration + 1, ' of ', MaxIterations)
        
        Rank, Frequency, Word, GridWords, AcrossRef, AcrossPos, DownRef, DownPos = GetData(WordFile, WordWorksheet, GridFile, GridWorksheet)
#         Rank, Word, Frequency = get_word_file_MILP("./gutenberg_lexicon.txt")
#         AcrossRef, AcrossPos, DownRef, DownPos, GridWords = get_grid_data_MILP(size = 7, display_grid = True)
        
        DefineModelData(Model, Rank, Frequency, Word, GridWords, AcrossRef, AcrossPos, DownRef, DownPos)
        Solver, Model = SetUpSolver(Model)
        
        Timer('Setup')
        DefineModel(Model)
        WriteModelToFile(WriteFile, Model)   # Write model to file, if required
        Timer('Create model')
        print('Calling solver...')
        Results, Model = CallSolver(Solver, Model)
        Timer('Solved');
        
        if (Results.solver.status == pyo.SolverStatus.ok) or (Solver._solver_model.SolCount >= 1):   # Feasible/optimal or found at least one feasible solution
            print('Solution for random seed:', Seed, '\n')
            Model.solutions.load_from(Results)   # Defer loading results until now, in case there is no solution to load
            WriteOutput(Model, Results)
            if StopOnFirst:   # Ignore iteration count and stop on first solution
                Finished = True
        else:
            print('No solution for random seed:', Seed, '(', Results.solver.termination_condition, ')', '\n')
        Iteration += 1        
        Seed += 1
        if Iteration >= MaxIterations:   # If not stopping on first iteration, then stop after specified number of iterations
            Finished = True
    Timer('Finish')
    WriteCheckpoints()

In [ ]:
# Data assumptions
Lexicon = 'gutenberg.xlsx'   # large.xlsx
Grid = 'grid-7-2.xlsx'
SampleSize = 5000   # Number of words to randomly select from WordFile. 0 means select all words

# Run options
MipGap = 100   # Highs: 100 (10000%) = stop on first feasible solution, or thereabouts; 0 = find optimum; CPLEX: 1 = first feasible, 0 = optimal; Gurobi: 100
SolutionLimit = 1   # Gurobi only, 1 = stop on first MIP solution
MaxIterations = 1   # Iterate random seeds, starting with RandomSeed and incrementing by 1 each iteration
StopOnFirst = True   # Stop on first solution, even if < MaxIterations
RandomSeed = 1
Direction = 1   # 1 = maximize, -1 = minimize

# Solver options
Neos = False
SolverName = 'gurobi_direct'
os.environ['NEOS_EMAIL'] = 'your.email@example.com'
Verbose = True
LoadSolution = False
TimeLimit = 1*3600   # seconds

# Model file
WriteFile = False
ModelFile = 'model-1.nl'   # Extensions: .gams .lp .nl

# Fixed
ModelName = 'Crossword creator - Model 1'
Checkpoints = []   # List of time checkpoints - Global Variable
WordWorksheet = 'Data'
GridWorksheet = 'Grid'
WordFile = os.path.join(os.getcwd() + '\lexicon', Lexicon)
GridFile = os.path.join(os.getcwd() + '\grid', Grid)
MaxWordLength = 7

In [ ]:
Main()

In [ ]:
Rank, Word, Frequency = get_word_file_MILP("./gutenberg_lexicon.txt")
AcrossRef, AcrossPos, DownRef, DownPos, GridWords = get_grid_data_MILP(size = 7, display_grid = True)

In [ ]:
Timer('Start');
Finished = False
Iteration = 0
Seed = RandomSeed

rnd.seed(Seed)
Model = pyo.ConcreteModel(name = ModelName)
Model.Engine = SolverName
Model.TimeLimit = TimeLimit

print(Model.name)   # Print the model name only once
print('Grid:    ', GridFile.rsplit('\\', 1)[-1])
print('Lexicon: ', WordFile.rsplit('\\', 1)[-1])

DefineModelData(Model, Rank, Frequency, Word, GridWords, AcrossRef, AcrossPos, DownRef, DownPos)
Solver, Model = SetUpSolver(Model)

Timer('Setup')

DefineModel(Model)
WriteModelToFile(WriteFile, Model)   # Write model to file, if required

Timer('Create model')

print('Calling solver...')
Results, Model = CallSolver(Solver, Model)

Timer('Solved');

if (Results.solver.status == pyo.SolverStatus.ok) or (Solver._solver_model.SolCount >= 1):   # Feasible/optimal or found at least one feasible solution
    print('Solution for random seed:', Seed, '\n')
    Model.solutions.load_from(Results)   # Defer loading results until now, in case there is no solution to load
    WriteOutput(Model, Results)
    if StopOnFirst:   # Ignore iteration count and stop on first solution
        Finished = True
else:
    print('No solution for random seed:', Seed, '(', Results.solver.termination_condition, ')', '\n')

Timer('Finish')
WriteCheckpoints()